# People and places in Post Office Directories: changes over time

From the 'Run' menu, select 'Run all cells'.

In [7]:
%pip install ipywidgets ipyleaflet

In [1]:
import pandas as pd
import ipywidgets as widgets
from ipyleaflet import FeatureGroup
from ipyleaflet import Map, Marker, MarkerCluster, WidgetControl, Heatmap, CircleMarker, FeatureGroup, LayerGroup
import time
import altair as alt

In [2]:
df = pd.read_csv("directories-place-year.csv", parse_dates=["year"])

In [3]:
df_places = df.copy().loc[df["type"].notnull()]

In [4]:
m_heat = Map(zoom=7, center=(-42, 147), layout=widgets.Layout(width='80%', height='600px'))

def add_locations(df):

    layer = LayerGroup()
    for place, people in df.groupby(["id", "latitude", "longitude"]):
        circle_marker = CircleMarker()
        circle_marker.location = (float(place[1]), float(place[2]))
        circle_marker.radius = people.shape[0] * 5
        circle_marker.color = "red"
        circle_marker.fill_color = "red"
        circle_marker.opacity = 0.8
        circle_marker.stroke = 0
        layer.add(circle_marker)
    try:
        m_heat.substitute(m_heat.layers[1], layer)
    except IndexError:
        m_heat.add(layer)

def change_heat_data(change):
    #print(change.new)
    #print(gdf.loc[df_places["year"] == str(change.new)].shape)
    #print([l.name for l in  m.layers])
    data = df_places.loc[df_places["year"] == str(change.new)]
    add_locations(data)

def animate():
    for year in range(df_places.year.dt.year.min() + 1, df_places.year.dt.year.max() + 1, 1):
        heat_slider.value = year
        time.sleep(2)
        
                         
play = widgets.Play(
    value=df_places.year.dt.year.min(),
    min=df_places.year.dt.year.min(),
    max=df_places.year.dt.year.max(),
    step=1,
    interval=1000,
    description="Press play",
    disabled=False
)
heat_slider = widgets.IntSlider(min=df_places.year.dt.year.min(), max=df_places.year.dt.year.max(), value=1, description='Year', continuous_update=False,)
heat_slider.observe(change_heat_data, names='value')
widgets.jslink((play, 'value'), (heat_slider, 'value'))

play_widget = widgets.HBox([play, heat_slider])
m_heat.add_control(WidgetControl(widget=play_widget, position='bottomleft'))
add_locations(df_places.loc[df_places["year"] == str(df_places.year.dt.year.min())])

m_heat

Map(center=[-42, 147], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_t…

In [8]:
m = Map(zoom=7, center=(-42, 147), layout=widgets.Layout(width='80%', height='600px'))

def add_markers(df):
    markers = []
    
    for point in df.itertuples():
        marker = Marker(location=(float(point.latitude), float(point.longitude)))
        message = widgets.HTML()
        message.value = f"<a href='https://data.katebagnall.com/s/tasmania/item/{point.person_id}' target='_blank'>{point.person_name}</a>"
        marker.popup = message
        markers.append(marker)
    marker_cluster = MarkerCluster(markers=markers)
    try:
        m.substitute(m.layers[1], marker_cluster)
    except IndexError:
        m.add(marker_cluster)

def change_data(change):
    #print(change.new)
    #print(gdf.loc[df_places["year"] == str(change.new)].shape)
    #print([l.name for l in  m.layers])
    data = df_places.loc[df_places["year"] == str(change.new)]
    add_markers(data)
                         


    
slider = widgets.IntSlider(min=df_places.year.dt.year.min(), max=df_places.year.dt.year.max(), value=1, description='Year', continuous_update=False,)
slider.observe(change_data, names='value')
m.add_control(WidgetControl(widget=slider, position='bottomleft'))
add_markers(df_places.loc[df_places["year"] == str(df_places.year.dt.year.min())])
m

Map(center=[-42, 147], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_t…